# EC Orphan Drug Records

Source of data: https://ec.europa.eu/health/documents/community-register/html/reg_od_act.htm?sort=n

In [2]:
import os

import numpy as np
import pandas as pd

path = os.path.dirname(os.getcwd())
data = pd.read_csv(path + '/data/ec_orphan_designations.csv', header=2)
data

,EU #,Product,Indication,Sponsor,Designation date,Tradename - EU product # - Implemented on
0,EU/3/20/2351,Adeno-associated virus serotype 5 containing t...,Treatment of RDH12 mutation associated retinal...,MeiraGTx B.V.,19 Oct 2020,-
1,EU/3/20/2350,"Poly(oxy-1,2-ethanediyl), alpha-hydro-omega-me...",Treatment of hypoparathyroidism,Ascendis Pharma Bone Diseases A/S,19 Oct 2020,-
2,EU/3/20/2349,Miglustat,Treatment of neuronal ceroid lipofuscinosis,Theranexus S.A.S.,19 Oct 2020,-
3,EU/3/20/2348,"Poly(oxy-1,2-ethanediyl), alpha-(carboxymethyl...",Treatment of homocystinuria,Aeglea Biotherapeutics UK Limited,19 Oct 2020,-
4,EU/3/20/2347,Trehalose,Treatment of neuronal ceroid lipofuscinosis,Theranexus S.A.S.,19 Oct 2020,-
...,...,...,...,...,...,...
1735,EU/3/01/034,Gusperimus trihydrochloride,Treatment of Wegener’s granulomatosis,Nordic Group B.V.,29 Mar 2001,-
1736,EU/3/01/028,Inolimomab,Treatment of Graft versus Host Disease,Elsalys Biotech SA,05 Mar 2001,-
1737,EU/3/01/026,L-Lysine-N-acetyl-L-cysteinate,Treatment of cystic fibrosis,LABORATOIRES SMB SA,14 Feb 2001,-
1738,EU/3/00/013,Ethyl Eicosopentaenoate,Treatment of Huntington's disease,Amarin Neuroscience Limited,29 Dec 2000,-


## 1. Number of designations each year
(exclude 2000 and 2020, because data is incomplete)

In [15]:
import plotly.colors
import plotly.graph_objects as go

# Create container for time data
time_evolution = pd.DataFrame(columns={'Year', 'Number of designations'})

# Group data by year
years = pd.to_datetime(
    data['Designation date']).apply(lambda x: x.year)
for year in years.dropna().unique():
    # Ignore 2000 and 2020
    if (year == 2000) or (year == 2020):
        continue

    # Get outcomes for that year
    mask = years == year
    n_designations = years[mask].count()
    
    # Append to main container
    time_evolution = time_evolution.append(pd.DataFrame({
        'Year': [year],
        'Number of designations': [n_designations]}))

# Sort by time
time_evolution.sort_values(by='Year', inplace=True)

# Create line plot
fig = go.Figure()
colour = plotly.colors.qualitative.Plotly[0]

fig.add_trace(go.Scatter(
    x=time_evolution['Year'],
    y=time_evolution['Number of designations'],
    marker=dict(
        line=dict(color=colour, width=1))))

# Set axes and template
fig.update_layout(
    template="plotly_white",
    xaxis_title='Year',
    yaxis_title='Number of designations')

fig.show()

**Figure 1:** EC's record of orphan drug designations over time

Note: There is a lag from EMA positive numbers to EC numbers, likely because date of first decision and date of designation do not coincide.

## 2 Number of applications per active substance
The substance names are transformed to lower case, but other than that there is no processing or matching of the names at this point

### 2.1 Number of single applications versus number of multiple applications

In [19]:
# Create container for name, number of applications
substance_applications = pd.DataFrame(columns={'Product', 'Number of designations'})

# Get the number of applications
temp = data.copy()
temp['Product'] = temp['Product'].str.lower()
substances = temp['Product'].unique()
for substance in substances:
    # Get number
    mask = temp['Product'] == substance
    n_applications = len(temp[mask])

    # Append to container
    substance_applications = substance_applications.append(pd.DataFrame({
        'Product': [substance],
        'Number of designations': [n_applications]}))

# Compute number of single and multi applications
mask = substance_applications['Number of designations'] == 1
n_single = len(substance_applications[mask])
mask = substance_applications['Number of designations'] > 1
n_multi = len(substance_applications[mask])

# Create figure
fig = go.Figure()
colours = plotly.colors.qualitative.Plotly

fig.add_trace(go.Pie(
    labels=['Single designation', 'Multiple designations'],
    values=[n_single, n_multi],
    sort=False,
    pull=[0, 0.2],
    marker=dict(
        colors=colours[:2], 
        line=dict(color='#000000', width=1))))
fig.show()

**Figure 2:** Number of orphan designation per product (combination of substances count as distinct, so do distinct names for the same substance).

### 2.2 Distribution among multiple applications

In [21]:
# Sort by number of applications
substance_applications.sort_values(by='Number of designations', inplace=True)

# Get number of applications
n_applications = []
n_substances = []
for n_application in substance_applications['Number of designations'].unique():
    # Skip if only one application
    if n_application <= 1:
        continue

    # Get number of substances
    mask = substance_applications['Number of designations'] == n_application
    number = len(substance_applications[mask])

    # Append to containers
    n_applications.append(n_application)
    n_substances.append(number)

# Accumulate >= 5 applications
n_applications = np.array(n_applications)
n_substances = np.array(n_substances)
mask = n_applications >= 5
n_substances_five_or_more = np.sum(n_substances[mask])
n_applications = n_applications[~mask]
n_substances = list(n_substances[~mask])

# Create figure
fig = go.Figure()
colours = plotly.colors.qualitative.Plotly
n_colours = len(n_substances) + 1

fig.add_trace(go.Pie(
    labels=[
        '%s designations' % str(n) for n in n_applications] + ['5 and more designations'],
    values=n_substances + [n_substances_five_or_more],
    sort=False,
    pull=[0, 0, 0, 0.2],
    marker=dict(
        colors=colours[:n_colours], 
        line=dict(color='#000000', width=1))))
fig.show()

**Figure 4:** Number of orphan status applications per active substance with more than one application(combination of substances count as distinct, so do distinct names for the same substance).